In [19]:
# 라이브러리 불러오기
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import pearsonr, bartlett, levene, wilcoxon, mannwhitneyu, ttest_ind, f_oneway, kruskal
from scipy.stats import shapiro, anderson, kstest, normaltest, jarque_bera
from statsmodels.formula.api import ols
from statsmodels.stats.anova import anova_lm
from statsmodels.stats.multicomp import pairwise_tukeyhsd
from statsmodels.stats.outliers_influence import variance_inflation_factor

In [20]:
plt.rc("font", family="Malgun Gothic")

In [21]:
df = pd.read_csv(r'C:\Users\User\Desktop\DigitalHana_Project2\Data\HanaCard_Data.csv', encoding = 'cp949')
df.head()

,회원번호,남녀구분코드,연령,소지여부_신용,입회일자_신용,입회경과개월수_신용,탈회여부,회원여부_연체,수신거부여부_TM,수신거부여부_DM,...,캠페인접촉건수_R12M,캠페인접촉일수_R12M,캠페인접촉월수_R12M,접촉후경과월_R12M,보험콜대상월수_R12M,보험콜대상횟수_R12M,보험콜통화횟수_R12M,보험콜배정후경과월_R12M,보험콜통화후경과월_R12M,고객구분
0,YNtSN9Oa,2,64,1,20140709,54,1,0,0,0,...,3,3,2,5,5,5,9,0,0,21.Active_연속
1,Rijqdlzu,1,24,1,20170412,21,0,0,0,0,...,0,0,0,12,5,5,14,2,2,21.Active_연속
2,NOsf3TZS,2,43,1,20170426,21,1,0,1,0,...,0,0,0,12,0,0,0,12,12,21.Active_연속
3,7IPZGTZu,2,79,1,20141128,50,0,0,0,0,...,16,15,5,1,3,3,6,0,0,21.Active_연속
4,pEEsUF2u,2,82,1,20120905,76,0,0,0,0,...,4,4,3,1,1,1,4,3,2,21.Active_연속


In [22]:
df.shape
# 데이터 갯수 : 300000개
# Column 갯수 : 302개
# Key 데이터 : 회원번호, Target 데이터 : 탈회여부

(300000, 302)

In [23]:
# Column들의 종류를 나누기
# '거주시도명', '거주시군구명', '직장시도명', '직장시군구명'의 경우는 비식별화 처리가 되어서 사용이 어렵기 때문에 제외
# 특이사항1 Column : '회원번호'(개인 구분용), '입회일자_신용'(날짜데이터)
# 특이사항2 Column : '납부_렌탈료이용금액', '컨택건수_CA_TM_B0M', '컨택건수_CA_EM_B0M', '컨택건수_리볼빙_EM_B0M', '컨택건수_리볼빙_청구서_B0M', '컨택건수_카드론_인터넷_B0M', '컨택건수_CA_인터넷_B0M', '컨택건수_리볼빙_인터넷_B0M', '컨택건수_CA_EM_R6M', '컨택건수_리볼빙_EM_R6M', '컨택건수_리볼빙_청구서_R6M', '컨택건수_카드론_인터넷_R6M', '컨택건수_CA_인터넷_R6M', '컨택건수_리볼빙_인터넷_R6M' -> 30만개 모든 값이 0
numeric_discrete = ['연령', '입회경과개월수_신용', '탈회횟수_누적', '탈회횟수_발급1년이내', '유효카드수_신용체크', '유효카드수_신용', '유효카드수_체크', '이용금액_R3M_신용체크', '이용금액_R3M_신용', '이용금액_R3M_체크', '_1순위카드이용금액', '_1순위카드이용건수', '기본연회비_B0M', '카드이용한도금액', '이용건수_신용_B0M', '이용건수_신판_B0M', '이용건수_일시불_B0M', '이용건수_할부_B0M', '이용건수_할부_유이자_B0M', '이용건수_할부_무이자_B0M', '이용건수_부분무이자_B0M', '이용건수_CA_B0M', '이용건수_체크_B0M', '이용건수_카드론_B0M', '이용금액_신용_B0M', '이용금액_신판_B0M', '이용금액_일시불_B0M', '이용금액_할부_B0M', '이용금액_할부_유이자_B0M', '이용금액_할부_무이자_B0M', '이용금액_부분무이자_B0M', '이용금액_CA_B0M', '이용금액_체크_B0M', '이용금액_카드론_B0M', '이용후경과월_신용', '이용후경과월_신판', '이용후경과월_일시불', '이용후경과월_할부', '이용후경과월_할부_유이자', '이용후경과월_할부_무이자', '이용후경과월_부분무이자', '이용후경과월_CA', '이용후경과월_체크', '이용후경과월_카드론', '이용건수_신용_R12M', '이용건수_신판_R12M', '이용건수_일시불_R12M', '이용건수_할부_R12M', '이용건수_할부_유이자_R12M', '이용건수_할부_무이자_R12M', '이용건수_부분무이자_R12M', '이용건수_CA_R12M', '이용건수_체크_R12M', '이용건수_카드론_R12M', '이용금액_신용_R12M', '이용금액_신판_R12M', '이용금액_일시불_R12M', '이용금액_할부_R12M', '이용금액_할부_유이자_R12M', '이용금액_할부_무이자_R12M', '이용금액_부분무이자_R12M', '이용금액_CA_R12M', '이용금액_체크_R12M', '이용금액_카드론_R12M', '이용개월수_신용_R12M', '이용개월수_신판_R12M', '이용개월수_일시불_R12M', '이용개월수_할부_R12M', '이용개월수_할부_유이자_R12M', '이용개월수_할부_무이자_R12M', '이용개월수_부분무이자_R12M', '이용개월수_CA_R12M', '이용개월수_체크_R12M', '이용개월수_카드론_R12M', '이용건수_신용_R6M', '이용건수_신판_R6M', '이용건수_일시불_R6M', '이용건수_할부_R6M', '이용건수_할부_유이자_R6M', '이용건수_할부_무이자_R6M', '이용건수_CA_R6M', '이용건수_체크_R6M', '이용건수_카드론_R6M', '이용금액_신용_R6M', '이용금액_신판_R6M', '이용금액_일시불_R6M', '이용금액_할부_R6M', '이용금액_할부_유이자_R6M', '이용금액_할부_무이자_R6M', '이용금액_부분무이자_R6M', '이용금액_CA_R6M', '이용금액_체크_R6M', '이용금액_카드론_R6M', '이용개월수_신용_R6M', '이용개월수_신판_R6M', '이용개월수_일시불_R6M', '이용개월수_할부_R6M', '이용개월수_할부_유이자_R6M', '이용개월수_할부_무이자_R6M', '이용개월수_부분무이자_R6M', '이용개월수_CA_R6M', '이용개월수_체크_R6M', '이용개월수_카드론_R6M', '이용건수_신용_R3M', '이용건수_신판_R3M', '이용건수_일시불_R3M', '이용건수_할부_R3M', '이용건수_할부_유이자_R3M', '이용건수_할부_무이자_R3M', '이용건수_부분무이자_R3M', '이용건수_CA_R3M', '이용건수_체크_R3M', '이용건수_카드론_R3M', '이용금액_신용_R3M', '이용금액_신판_R3M', '이용금액_일시불_R3M', '이용금액_할부_R3M', '이용금액_할부_유이자_R3M', '이용금액_할부_무이자_R3M', '이용금액_부분무이자_R3M', '이용금액_CA_R3M', '이용금액_체크_R3M', '이용금액_카드론_R3M', '이용개월수_신용_R3M', '이용개월수_신판_R3M', '이용개월수_일시불_R3M', '이용개월수_할부_R3M', '이용개월수_할부_유이자_R3M', '이용개월수_할부_무이자_R3M', '이용개월수_부분무이자_R3M', '이용개월수_CA_R3M', '이용개월수_체크_R3M', '이용개월수_카드론_R3M', '이용가맹점수', '이용금액_업종기준', '이용금액_쇼핑', '이용금액_요식', '이용금액_교통', '이용금액_의료', '이용금액_납부', '이용금액_교육', '이용금액_여유생활', '이용금액_사교활동', '이용금액_일상생활', '이용금액_해외', '쇼핑_도소매_이용금액', '쇼핑_백화점_이용금액', '쇼핑_마트_이용금액', '쇼핑_슈퍼마켓_이용금액', '쇼핑_편의점_이용금액', '쇼핑_아울렛_이용금액', '쇼핑_온라인_이용금액', '쇼핑_기타_이용금액', '쇼핑_전체_이용금액', '교통_주유이용금액', '교통_정비이용금액', '교통_통행료이용금액', '교통_버스지하철이용금액', '교통_택시이용금액', '교통_철도버스이용금액', '교통_전체이용금액', '여유_운동이용금액', '여유_Pet이용금액', '여유_공연이용금액', '여유_공원이용금액', '여유_숙박이용금액', '여유_여행이용금액', '여유_항공이용금액', '여유_기타이용금액', '여유_전체이용금액', '납부_통신비이용금액', '납부_관리비이용금액', '납부_가스전기료이용금액', '납부_보험료이용금액', '납부_유선방송이용금액', '납부_건강연금이용금액', '납부_기타이용금액', '납부_전체이용금액', '할부건수_3M_R12M', '할부건수_6M_R12M', '할부건수_12M_R12M', 'RP건수_B0M', 'RP금액_B0M', '증감_RP건수_전월', '증감_RP유형건수_전월', '이용개월수_온라인_R6M', '이용개월수_오프라인_R6M', '이용건수_오프라인_R6M', '이용건수_온라인_B0M', '이용건수_오프라인_B0M', '이용개월수_페이_온라인_R6M', '이용개월수_페이_오프라인_R6M', '이용금액_페이_온라인_R6M', '이용건수_페이_온라인_R6M', '이용건수_페이_오프라인_R6M', '이용건수_페이_오프라인_R3M', '이용건수_페이_온라인_B0M', '이용건수_페이_오프라인_B0M', '이용개월수_간편결제_R6M', '이용개월수_전체_R6M', '이용개월수_전체_R3M', '이용개월수_결제일_R6M', '청구금액_B0', '청구금액_R3M', '청구금액_R6M', '할인건수_R3M', '할인금액_R3M', '할인건수_B0M', '할인금액_B0M', '상환개월수_R12M', '상환개월수_결제일_R12M', '상환개월수_결제일_R6M', '상환개월수_결제일_R3M', '연체건수_R12M', '연체건수_R6M', '연체건수_R3M', '혜택수혜금액', '컨택건수_카드론_TM_B0M', '컨택건수_리볼빙_TM_B0M', '컨택건수_이용유도_TM_B0M', '컨택건수_신용발급_TM_B0M', '컨택건수_부대서비스_TM_B0M', '컨택건수_포인트소진_TM_B0M', '컨택건수_보험_TM_B0M', '컨택건수_카드론_LMS_B0M', '컨택건수_CA_LMS_B0M', '컨택건수_리볼빙_LMS_B0M', '컨택건수_이용유도_LMS_B0M', '컨택건수_카드론_EM_B0M', '컨택건수_이용유도_EM_B0M', '컨택건수_카드론_청구서_B0M', '컨택건수_CA_청구서_B0M', '컨택건수_이용유도_청구서_B0M', '컨택건수_이용유도_인터넷_B0M', '컨택건수_카드론_TM_R6M', '컨택건수_리볼빙_TM_R6M', '컨택건수_CA_TM_R6M', '컨택건수_이용유도_TM_R6M', '컨택건수_신용발급_TM_R6M', '컨택건수_부대서비스_TM_R6M', '컨택건수_포인트소진_TM_R6M', '컨택건수_보험_TM_R6M', '컨택건수_카드론_LMS_R6M', '컨택건수_CA_LMS_R6M', '컨택건수_리볼빙_LMS_R6M', '컨택건수_이용유도_LMS_R6M', '컨택건수_카드론_EM_R6M', '컨택건수_이용유도_EM_R6M', '컨택건수_카드론_청구서_R6M', '컨택건수_CA_청구서_R6M', '컨택건수_이용유도_청구서_R6M', '컨택건수_채권_B0M', '컨택건수_FDS_B0M', '컨택건수_채권_R6M', '컨택건수_FDS_R6M', '캠페인접촉건수_R12M', '캠페인접촉일수_R12M', '캠페인접촉월수_R12M', '접촉후경과월_R12M', '보험콜대상월수_R12M', '보험콜대상횟수_R12M', '보험콜통화횟수_R12M', '보험콜배정후경과월_R12M', '보험콜통화후경과월_R12M']
numeric_continuos = []
categorical_order = ['Life_Stage', '이용금액대']
categorical_nonorder = ['남녀구분코드', '소지여부_신용', '탈회여부', '회원여부_연체', '수신거부여부_TM', '수신거부여부_DM', '수신거부여부_메일', '수신거부여부_SMS', '자격기준', '자격구분상세', '연회비청구구분_B0M', 'BL여부_R12M', '마케팅동의여부', '_1순위카드ID', '_1순위신용체크구분', '고객구분']

In [24]:
# 활용할 수 없는 Column들을 삭제
df.drop(columns = ['거주시도명', '거주시군구명', '직장시도명', '직장시군구명', '납부_렌탈료이용금액', '컨택건수_CA_TM_B0M', '컨택건수_CA_EM_B0M', '컨택건수_리볼빙_EM_B0M', '컨택건수_리볼빙_청구서_B0M', '컨택건수_카드론_인터넷_B0M', '컨택건수_CA_인터넷_B0M', '컨택건수_리볼빙_인터넷_B0M', '컨택건수_CA_EM_R6M', '컨택건수_리볼빙_EM_R6M', '컨택건수_리볼빙_청구서_R6M', '컨택건수_카드론_인터넷_R6M', '컨택건수_CA_인터넷_R6M', '컨택건수_리볼빙_인터넷_R6M'], inplace = True)

In [25]:
df['Life_Stage'] = df['Life_Stage'].map(lambda x : int(x[0]))
df['이용금액대'] = df['이용금액대'].map(lambda x : {9 : 0, 5 : 1, 4 : 2, 3 : 3, 2 : 4, 1 : 5}[int(x[:2])])

# 신용카드 추천 시스템

In [26]:
# 그룹화 시킬 기준
# 그룹화 시킬 수 있는 전체 기준 : ['회원번호', '자격기준', '자격구분상세', '연회비청구구분_B0M', '_1순위신용체크구분', 'Life_Stage', '이용금액대', '고객구분', 남녀구분코드, 소지여부_신용, 회원여부_연체, 수신거부여부_TM, 수신거부여부_DM, 수신거부여부_메일, 수신거부여부_SNS, BL여부_R12M, 마케팅동의여부, _1순위카드ID]
# 그룹화 시킬 수 있는 전체 기준 : 남녀구분코드, 소지여부_신용, 회원여부_연체, 수신거부여부_TM, 수신거부여부_DM, 수신거부여부_메일, 수신거부여부_SNS, BL여부_R12M, 마케팅동의여부, _1순위카드ID
# 확실한 것들 : 남녀구분코드, 연령 or Life_Stage, 이용금액대, 연회비청구구분_B0M, 소지여부_신용, 회원여부_연체, BL여부_R12M
# 애매한 것들 : 자격구분상세(?)

In [27]:
# '남녀구분코드', 'Life_Stage', '이용금액대', '연회비청구구분_B0M', '소지여부_신용', '회원여부_연체', 'BL여부_R12M', '자격구분상세'으로 그룹화했을 때 총 몇 개의 그룹이 나오는지 확인
len(df.groupby(by = ['남녀구분코드', 'Life_Stage', '이용금액대', '연회비청구구분_B0M', '소지여부_신용', '회원여부_연체', 'BL여부_R12M', '자격구분상세']))

9844

In [28]:
# 그룹중에 갯수가 0개인 것을 확인
for group_name, group in df.groupby(by = ['남녀구분코드', 'Life_Stage', '이용금액대', '연회비청구구분_B0M', '소지여부_신용', '회원여부_연체', 'BL여부_R12M', '자격구분상세']):
    if len(group) == 0:
        print(group_name)

## 1. 고객 프로파일

In [29]:
# 1. 소비패턴 프로파일(user profile for consumption pattern, UPC)
# '이용금액_업종기준', '쇼핑_도소매_이용금액', '쇼핑_전체_이용금액', '이용금액_쇼핑', '교통_전체이용금액', '이용금액_교통', '여유_전체이용금액', '이용금액_여유생활'
# '이용금액_요식', '이용금액_의료', '이용금액_납부', '이용금액_교육', '이용금액_사교활동', '이용금액_일상생활', '이용금액_해외', '쇼핑_백화점_이용금액', '쇼핑_마트_이용금액', '쇼핑_슈퍼마켓_이용금액', '쇼핑_편의점_이용금액', '쇼핑_아울렛_이용금액', '쇼핑_온라인_이용금액', '쇼핑_기타_이용금액', '교통_주유이용금액', '교통_정비이용금액', '교통_통행료이용금액', '교통_버스지하철이용금액', '교통_택시이용금액', '교통_철도버스이용금액', '여유_운동이용금액', '여유_Pet이용금액', '여유_공연이용금액', '여유_공원이용금액', '여유_숙박이용금액', '여유_여행이용금액', '여유_항공이용금액', '여유_기타이용금액'
# 2. 거래성향 프로파일(user profile for propensity to trade, UPP)
# (+?) 할부 여부, 카드론 사용 여부 등
# 연회비청구구분_B0M’, ‘소지여부_신용’, ‘회원여부_연체’, ‘BL여부_R12M’
# 3. 인구통계학 프로파일(user profile for demographics, UPD)
# '남녀구분코드', '연령', '자격구분상세'
# 4. 거래방법 프로파일(user profile for trading method, UPT)
# (+?) 온라인 , 오프라인 결제 건수 등
# 5. 마케팅 프로파일(user profile for marketing, UPM)

In [30]:
# 5개의 프로파일에 해당하는 변수들 구분
profile1 = ['이용금액_업종기준', '이용금액_요식', '이용금액_의료', '이용금액_납부', '이용금액_교육', '이용금액_사교활동', '이용금액_일상생활', '이용금액_해외', '쇼핑_백화점_이용금액', '쇼핑_마트_이용금액', '쇼핑_슈퍼마켓_이용금액', '쇼핑_편의점_이용금액', '쇼핑_아울렛_이용금액', '쇼핑_온라인_이용금액', '쇼핑_기타_이용금액', '교통_주유이용금액', '교통_정비이용금액', '교통_통행료이용금액', '교통_버스지하철이용금액', '교통_택시이용금액', '교통_철도버스이용금액', '여유_운동이용금액', '여유_Pet이용금액', '여유_공연이용금액', '여유_공원이용금액', '여유_숙박이용금액', '여유_여행이용금액', '여유_항공이용금액', '여유_기타이용금액', '납부_통신비이용금액', '납부_관리비이용금액', '납부_가스전기료이용금액', '납부_보험료이용금액', '납부_유선방송이용금액', '납부_건강연금이용금액', '납부_기타이용금액', 'RP금액_B0M']
profile2 = ['연회비청구구분_B0M', '소지여부_신용', '회원여부_연체', 'BL여부_R12M', '카드이용한도금액', '이용가맹점수', '이용금액_업종기준', '이용금액대', '혜택수혜금액', 'Life_Stage', '할부건수_3M_R12M', '할부건수_6M_R12M', '할부건수_12M_R12M']
profile3 = ['남녀구분코드', '연령', '자격구분상세']
profile4 = []
profile5 = []

# 고객구분이 연속인 사람(너무 많으면 Memory부족 문제가 생겨서 2.5만명으로 제한)과 간헐인 사람
df1 = df.loc[df['고객구분'] == '21.Active_연속'][:25000]
df2 = df.loc[df['고객구분'] == '22.Active_간헐']

### 1-1. 소비패턴 고객 네트워크

In [31]:
# 1. 소비패턴 고객 네트워크
# (1) 코사인 유사도 데이터프레임
df_network1 = pd.DataFrame(np.dot(df1[profile1], df2[profile1].T), index = df1[profile1].index, columns = df2[profile1].index)
df_network1.index.name = 'Active_연속'
df_network1.columns.name = 'Active_간헐'

df_network1 = df_network1.div(df1[profile1].apply(lambda x : np.linalg.norm(x), axis = 1), axis = 0)
df_network1 = df_network1.div(df2[profile1].apply(lambda x : np.linalg.norm(x), axis = 1), axis = 1)

In [52]:
df_network1

Active_간헐,29,31,33,35,46,49,54,57,60,69,...,299903,299909,299921,299925,299933,299956,299967,299975,299989,299996
Active_연속,,,,,,,,,,,,,,,,,,,,,
0,0.601894,0.435185,0.668462,0.515293,0.667737,0.547126,0.501236,0.466967,0.499866,0.435185,...,0.530392,0.436079,0.555939,0.435185,0.691567,0.615262,0.615444,0.980147,0.558255,0.691567
1,0.599909,0.430737,0.685937,0.510026,0.703391,0.542067,0.496113,0.460491,0.494757,0.430737,...,0.524971,0.431576,0.550257,0.430737,0.718387,0.608973,0.609154,0.969373,0.552549,0.718387
2,0.739873,0.543710,0.609595,0.658681,0.703900,0.689028,0.626232,0.594044,0.631876,0.543710,...,0.662659,0.544413,0.704305,0.543710,0.676285,0.768693,0.768922,0.963862,0.697470,0.676285
3,0.920919,0.644957,0.456053,0.763679,0.526605,0.956145,0.742846,0.673819,0.740815,0.644957,...,0.786056,0.645791,0.877408,0.644957,0.505946,0.911835,0.912106,0.526605,0.827349,0.505946
4,0.911874,0.675381,0.482061,0.823443,0.551446,0.903260,0.777888,0.706218,0.775761,0.816325,...,0.836658,0.676385,0.886585,0.675381,0.532305,0.954849,0.955133,0.551446,0.866377,0.532305
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
33545,0.558342,0.400892,0.944911,0.474687,0.981981,0.511238,0.461738,0.431727,0.460476,0.400892,...,0.488596,0.401758,0.512130,0.400892,0.995871,0.566778,0.566947,0.654654,0.514263,0.995871
33546,0.916868,0.669766,0.479234,0.805062,0.553372,0.945213,0.771421,0.720713,0.769312,0.685908,...,0.816293,0.670632,0.890758,0.679406,0.531662,0.948416,0.947192,0.550117,0.882466,0.531662
33547,0.911537,0.642656,0.452129,0.757108,0.522074,0.984686,0.736455,0.668021,0.734442,0.639408,...,0.780607,0.640235,0.900259,0.695294,0.501593,0.903990,0.904259,0.522074,0.842258,0.501593


In [32]:
# 1. 소비패턴 고객 네트워크
# (2) 코사인 유사도를 바탕으로 22.Active_간헐 고객들마다 21.Active_연속인 고객들 중 가장 유사한 고객 찾기
df_network1_res = df_network1.apply(lambda x : x.argmax())

In [53]:
df_network1_res

Active_간헐
29         8130
31        15932
33         3945
35         6848
46          285
          ...  
299956    12524
299967      247
299975      350
299989    13954
299996     9424
Length: 29035, dtype: int64

In [33]:
# 1. 소비패턴 고객 네트워크
# 소비패턴 프로파일 네트워크에서 벡터의 크기가 0(즉, 이용금액이 전부 0)이라서 패턴을 알 수 없는 간헐 고객의 수
df_network1_res[df_network1_res < 0]

Active_간헐
279      -1
323      -1
424      -1
463      -1
529      -1
         ..
299023   -1
299652   -1
299691   -1
299873   -1
299890   -1
Length: 1697, dtype: int64

In [34]:
# 소비패턴 프로파일 네트워크에서 벡터의 크기가 0(즉, 이용금액이 전부 0)이라서 패턴을 알 수 없는 간헐 고객의 예시
df.loc[279, profile1]

이용금액_업종기준       0
이용금액_요식         0
이용금액_의료         0
이용금액_납부         0
이용금액_교육         0
이용금액_사교활동       0
이용금액_일상생활       0
이용금액_해외         0
쇼핑_백화점_이용금액     0
쇼핑_마트_이용금액      0
쇼핑_슈퍼마켓_이용금액    0
쇼핑_편의점_이용금액     0
쇼핑_아울렛_이용금액     0
쇼핑_온라인_이용금액     0
쇼핑_기타_이용금액      0
교통_주유이용금액       0
교통_정비이용금액       0
교통_통행료이용금액      0
교통_버스지하철이용금액    0
교통_택시이용금액       0
교통_철도버스이용금액     0
여유_운동이용금액       0
여유_Pet이용금액      0
여유_공연이용금액       0
여유_공원이용금액       0
여유_숙박이용금액       0
여유_여행이용금액       0
여유_항공이용금액       0
여유_기타이용금액       0
납부_통신비이용금액      0
납부_관리비이용금액      0
납부_가스전기료이용금액    0
납부_보험료이용금액      0
납부_유선방송이용금액     0
납부_건강연금이용금액     0
납부_기타이용금액       0
RP금액_B0M        0
Name: 279, dtype: object

### 1-2. 거래성향 고객 네트워크

In [35]:
# 2. 거래성향 고객 네트워크
# (1) 코사인 유사도 데이터프레임
df1_eh = pd.get_dummies(data = df1[profile2], columns = ['연회비청구구분_B0M'], dtype = np.int64)
df2_eh = pd.get_dummies(data = df2[profile2], columns = ['연회비청구구분_B0M'], dtype = np.int64)
df_network2 = pd.DataFrame(np.dot(df1_eh, df2_eh.T), index = df1_eh.index, columns = df2_eh.index)
df_network2.index.name = 'Active_연속'
df_network2.columns.name = 'Active_간헐'

df_network2 = df_network2.div(df1_eh.apply(lambda x : np.linalg.norm(x), axis = 1), axis = 0)
df_network2 = df_network2.div(df2_eh.apply(lambda x : np.linalg.norm(x), axis = 1), axis = 1)

In [54]:
df_network2

Active_간헐,29,31,33,35,46,49,54,57,60,69,...,299903,299909,299921,299925,299933,299956,299967,299975,299989,299996
Active_연속,,,,,,,,,,,,,,,,,,,,,
0,0.695463,0.996952,0.994924,0.997128,0.995213,0.996667,0.998155,0.998840,0.995466,0.998207,...,0.999771,0.970637,0.999945,0.100822,0.994898,0.853270,0.999866,0.995185,0.995564,0.994921
1,0.739537,0.990027,0.986569,0.990348,0.987043,0.989516,0.992325,0.993808,0.987460,0.992432,...,0.996449,0.983910,0.997360,0.163575,0.986529,0.884556,0.996875,0.986997,0.987624,0.986566
2,0.689075,0.997612,0.995784,0.997768,0.996048,0.997358,0.998661,0.999234,0.996278,0.998706,...,0.999925,0.968477,0.999997,0.092003,0.995761,0.848623,0.999974,0.996023,0.996367,0.995782
3,0.780401,0.979228,0.974362,0.979693,0.975020,0.978495,0.982605,0.984875,0.975598,0.982767,...,0.989205,0.993204,0.990845,0.225276,0.974309,0.912140,0.989958,0.974955,0.975826,0.974359
4,0.946809,0.851457,0.839306,0.852663,0.840901,0.849570,0.860445,0.866820,0.842316,0.860890,...,0.880003,0.973389,0.885474,0.543910,0.839176,0.997229,0.882475,0.840745,0.842876,0.839299
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
33545,0.620187,0.999755,1.000000,0.999701,0.999997,0.999828,0.999219,0.998639,0.999988,0.999184,...,0.996860,0.941758,0.995871,0.000900,1.000000,0.796902,0.996433,0.999998,0.999982,1.000000
33546,0.871218,0.933566,0.925172,0.934390,0.926284,0.932275,0.939656,0.943907,0.927267,0.939955,...,0.952502,0.998952,0.955983,0.379820,0.925081,0.966402,0.954081,0.926175,0.927657,0.925167
33547,0.991639,0.532648,0.513257,0.534596,0.515778,0.529608,0.547280,0.557820,0.518020,0.548012,...,0.580088,0.772363,0.589534,0.858389,0.513051,0.927707,0.584341,0.515530,0.518910,0.513245


In [36]:
# 2. 거래성향 고객 네트워크
# (2) 코사인 유사도를 바탕으로 22.Active_간헐 고객들마다 21.Active_연속인 고객들 중 가장 유사한 고객 찾기
df_network2_res = df_network2.apply(lambda x : x.argmax())

In [55]:
df_network2_res

Active_간헐
29        13942
31        22851
33        18146
35        13487
46        23761
          ...  
299956    23795
299967    15901
299975    16225
299989     6542
299996    20911
Length: 29035, dtype: int64

In [37]:
# 2. 거래성향 고객 네트워크
# 거래성향 프로파일 네트워크에서 벡터의 크기가 0이라서 패턴을 알 수 없는 간헐 고객의 수
df_network2_res[df_network2_res < 0]

Series([], dtype: int64)

### 1-3. 인구통계학 고객 네트워크

In [38]:
# 3. 인구통계학 고객 네트워크
# df1(21.Active_연속)의 자격구분상세에 없는 고유값 찾기
set(pd.get_dummies(data = df['자격구분상세'], dtype = np.int64).columns) - set(pd.get_dummies(data = df1['자격구분상세'], dtype = np.int64).columns)

{'08.은행거래_하나저축은행거래'}

In [39]:
# 3. 인구통계학 고객 네트워크
# (1) 코사인 유사도 데이터프레임
df1_eh = pd.get_dummies(data = df1[profile3], columns = ['자격구분상세'], dtype = np.int64)
df2_eh = pd.get_dummies(data = df2[profile3], columns = ['자격구분상세'], dtype = np.int64)
df1_eh['자격구분상세_08.은행거래_하나저축은행거래'] = 0
df_network3 = pd.DataFrame(np.dot(df1_eh, df2_eh.T), index = df1_eh.index, columns = df2_eh.index)
df_network3.index.name = 'Active_연속'
df_network3.columns.name = 'Active_간헐'

df_network3 = df_network3.div(df1_eh.apply(lambda x : np.linalg.norm(x), axis = 1), axis = 0)
df_network3 = df_network3.div(df2_eh.apply(lambda x : np.linalg.norm(x), axis = 1), axis = 1)

In [56]:
df_network3

Active_간헐,29,31,33,35,46,49,54,57,60,69,...,299903,299909,299921,299925,299933,299956,299967,299975,299989,299996
Active_연속,,,,,,,,,,,,,,,,,,,,,
0,0.999618,0.999069,0.999615,0.999200,0.999520,0.999545,0.999679,0.999589,0.999576,0.999255,...,0.999557,0.999467,0.999628,0.999607,0.999504,0.999597,0.999632,0.999467,0.999351,0.999771
1,0.998700,0.998558,0.998701,0.998655,0.998672,0.998885,0.998949,0.998909,0.998696,0.998695,...,0.998689,0.998642,0.998695,0.998702,0.998663,0.998701,0.998689,0.998642,0.998761,0.998950
2,0.999177,0.999224,0.999180,0.999305,0.999181,0.999482,0.999514,0.999497,0.999191,0.999338,...,0.999189,0.999162,0.999165,0.999185,0.999176,0.999693,0.999155,0.999162,0.999391,0.999472
3,0.999712,0.998930,0.999706,0.999080,0.999574,0.999491,0.999665,0.999546,0.999648,0.999145,...,0.999622,0.999507,0.999729,0.999693,0.999895,0.999677,0.999739,0.999507,0.999256,0.999808
4,0.999722,0.998905,0.999716,0.999058,0.999578,0.999479,0.999659,0.999535,0.999655,0.999123,...,0.999629,0.999509,0.999741,0.999703,0.999557,0.999686,0.999752,0.999509,0.999238,0.999810
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
33545,0.999253,0.998704,0.999250,0.998834,0.999155,0.999180,0.999314,0.999223,0.999211,0.998890,...,0.999192,0.999102,0.999262,0.999242,0.999982,0.999231,0.999266,0.999102,0.998985,0.999405
33546,0.998980,0.999213,0.998985,0.999279,0.999016,0.999403,0.999403,0.999409,0.999011,0.999305,...,0.999015,0.999008,0.998961,0.998994,0.999015,0.999001,0.999387,0.999008,0.999345,0.999320
33547,0.999575,0.999107,0.999906,0.999231,0.999491,0.999553,0.999674,0.999593,0.999540,0.999284,...,0.999524,0.999442,0.999582,0.999566,0.999477,0.999558,0.999583,0.999442,0.999373,0.999747


In [40]:
# 3. 인구통계학 고객 네트워크
# (2) 코사인 유사도를 바탕으로 22.Active_간헐 고객들마다 21.Active_연속인 고객들 중 가장 유사한 고객 찾기
df_network3_res = df_network3.apply(lambda x : x.argmax())

In [57]:
df_network3_res

Active_간헐
29         2526
31          213
33          661
35           17
46         7170
          ...  
299956      531
299967     6895
299975      272
299989     3029
299996    16512
Length: 29035, dtype: int64

In [41]:
# 3. 인구통계학 고객 네트워크
# 인구통계학 프로파일 네트워크에서 벡터의 크기가 0이라서 패턴을 알 수 없는 간헐 고객의 수
df_network3_res[df_network3_res < 0]

Series([], dtype: int64)

## 2. 통합 네트워크 계산

In [42]:
# 3개의 고객 네트워크(소비패턴 고객 네트워크, 거래성향 고객 네트워크, 인구통계학 고객 네트워크)를 단순평균으로 통합 네트워크 계산
res = (df_network1 + df_network2 + df_network3) / 3

In [58]:
# 유사도가 가장 높은 순서대로 k명의 이웃 추출
k = 5
df_rc = res.apply(lambda x : x.sort_values(ascending = False)[:k].index)
df_rc.index.name = 'Active_연속_고객번호'

In [59]:
df_rc

Active_간헐,29,31,33,35,46,49,54,57,60,69,...,299903,299909,299921,299925,299933,299956,299967,299975,299989,299996
Active_연속_고객번호,,,,,,,,,,,,,,,,,,,,,
0,3600,21381,16852,9200,31857,17103,936,18828,2619,9947,...,29650,13009,10346,15062,29779,16173,1699,33324,1239,3026
1,17466,8351,6623,29133,23853,17317,12817,5815,7432,30757,...,5073,14864,16618,24156,16311,30369,21889,13282,20071,29779
2,20788,14412,5295,21294,31089,14425,2508,8227,13390,29174,...,9870,12400,13718,18263,3026,27038,27633,30137,32752,30081
3,15683,28815,24376,27645,509,824,12554,7531,9784,785,...,28335,18896,26468,12698,30081,29617,2258,28398,15492,12645
4,33181,14830,6462,31322,10555,21646,8295,13474,1690,33145,...,24855,27296,1242,28254,12645,33115,17956,22061,30190,12836


In [60]:
# 유사도가 높은 고객들의 1순위카드ID 확인
df_rc_res = df_rc.applymap(lambda x : df.loc[x, '_1순위카드ID'])
df_rc_res.index.name = 'Active_연속_1순위카드ID'

In [61]:
df_rc_res

Active_간헐,29,31,33,35,46,49,54,57,60,69,...,299903,299909,299921,299925,299933,299956,299967,299975,299989,299996
Active_연속_1순위카드ID,,,,,,,,,,,,,,,,,,,,,
0,98,1302,810,1555,656,1536,164,296,470,1739,...,156,1555,429,1229,811,136,1400,295,492,123
1,321,1303,810,136,1558,1145,164,355,324,1739,...,126,101,118,1554,809,136,313,216,1555,811
2,1529,186,810,136,158,270,1059,305,1068,1739,...,136,1186,1405,1405,123,1558,715,216,156,1353
3,136,311,1514,487,890,136,1555,270,308,1739,...,156,120,460,1555,1353,136,244,136,1536,890
4,143,360,1050,1493,124,791,101,1561,944,1739,...,311,492,1,299,890,897,1739,1493,313,240


In [62]:
# 각 간헐 고객의 1순위카드ID가 유사도가 높은 연속인 k명의 고객들의 1순위카드ID 종류에 포함되어있는지 확인
df_rc_res.apply(lambda x : (x == df.loc[x.name, '_1순위카드ID']).any())

Active_간헐
29        False
31        False
33        False
35        False
46        False
          ...  
299956    False
299967     True
299975    False
299989    False
299996    False
Length: 29035, dtype: bool

In [63]:
# 1순위카드ID가 유사도가 높은 연속인 k명의 고객들의 1순위카드ID 종류에 포함되어있는 간헐 고객의 수
df_rc_res.apply(lambda x : (x == df.loc[x.name, '_1순위카드ID']).any()).sum()

3624

In [64]:
3624 / 29035

0.12481487859479938